In [ ]:
from __future__ import absolute_import, division, print_function

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from pymor.discretizations.iosys import LTISystem
from pymor.reductors.bt import bt
from pymor.reductors.lti import irka

import logging
logging.getLogger('pymor.algorithms.gram_schmidt.gram_schmidt').setLevel(logging.ERROR)

In [ ]:
# dimension of the system
n = 100

# assemble A, B, and C
A = np.zeros((n, n))
a = n * (n - 1)
b = (n - 1) ** 2
A[0, 0] = -2 * a
A[0, 1] = 2 * b
for i in range(1, n - 1):
    A[i, i - 1] = b
    A[i, i] = -2 * b
    A[i, i + 1] = b
A[-1, -1] = -2 * a
A[-1, -2] = 2 * b

B = np.zeros((n, 1))
B[0, 0] = 2 * (n - 1)

C = np.zeros((1, n))
C[0, n - 1] = 1

In [ ]:
# LTI system
lti = LTISystem.from_matrices(A, B, C)

print('n = {}'.format(lti.n))
print('m = {}'.format(lti.m))
print('p = {}'.format(lti.p))

In [ ]:
# System poles
lti.compute_poles()
poles = lti._poles
fig, ax = plt.subplots()
ax.plot(poles.real, poles.imag, '.')
ax.set_title('System poles')
plt.show()

In [ ]:
# Bode plot of the full model
w = np.logspace(-2, 3, 100)
lti.bode(w)
fig, ax = LTISystem.mag_plot(lti)
ax.set_title('Bode plot of the full model')
plt.show()

In [ ]:
# Hankel singular values
lti.compute_sv_U_V('lyap')
fig, ax = plt.subplots()
ax.semilogy(range(1, len(lti._sv['lyap']) + 1), lti._sv['lyap'], '.-')
ax.set_title('Hankel singular values')
plt.show()

In [ ]:
# Norms of the system
print('H_2-norm of the full model:   {}'.format(lti.norm()))
print('H_inf-norm of the full model: {}'.format(lti.norm('Hinf')))

In [ ]:
# Balanced Truncation
r = 5
rom_bt, _, _ = bt(lti, r, me_solver='slycot')

In [ ]:
bla = -lti

In [ ]:
bla.D

In [ ]:
bla.C

In [ ]:
spam = lti - rom_bt

In [ ]:
print('H_2-norm of the BT ROM:       {}'.format(rom_bt.norm()))
print('H_inf-norm of the BT ROM:     {}'.format(rom_bt.norm('Hinf')))
err_bt = lti - rom_bt
err_bt.compute_gramian('lyap', 'cf', me_solver='slycot')
print('H_2-error for the BT ROM:     {}'.format(err_bt.norm()))
print('H_inf-error for the BT ROM:   {}'.format(err_bt.norm('Hinf')))

In [ ]:
# Bode plot of the full and BT reduced model
rom_bt.bode(w)
fig, ax = LTISystem.mag_plot((lti, rom_bt))
ax.set_title('Bode plot of the full and BT reduced model')
plt.show()

In [ ]:
# Bode plot of the BT error system
err_bt.bode(w)
fig, ax = LTISystem.mag_plot(err_bt)
ax.set_title('Bode plot of the BT error system')
plt.show()

In [ ]:
# Iterative Rational Krylov Algorithm
sigma = np.logspace(-1, 3, r)
tol = 1e-4
maxit = 100
rom_irka, _, reduction_data_irka = irka(lti, r, sigma, tol=tol, maxit=maxit, verbose=True, compute_errors=True)

In [ ]:
# print(reduction_data_irka['dist'])
tmp = map(np.min, reduction_data_irka['dist'])
# print(tmp)
fig, ax = plt.subplots()
ax.semilogy(tmp, '.-')
ax.set_title('Distances between shifts in IRKA iterations')
plt.show()

In [ ]:
print('H_2-norm of the IRKA ROM:     {}'.format(rom_irka.norm()))
print('H_inf-norm of the IRKA ROM:   {}'.format(rom_irka.norm('Hinf')))
err_irka = lti - rom_irka
print('H_2-error for the IRKA ROM:   {}'.format(err_irka.norm()))
print('H_inf-error for the IRKA ROM: {}'.format(err_irka.norm('Hinf')))

In [ ]:
# Bode plot of the full and IRKA reduced model
rom_irka.bode(w)
fig, ax = LTISystem.mag_plot((lti, rom_irka))
ax.set_title('Bode plot of the full and IRKA reduced model')
plt.show()

In [ ]:
# Bode plot of the IRKA error system
err_irka.bode(w)
fig, ax = LTISystem.mag_plot(err_irka)
ax.set_title('Bode plot of the IRKA error system')
plt.show()